## Part 2: Segmenting and Clustering Neighborhoods in Toronto
* Part 2 of Week 3 Assignment
* Task: Get the latitude and the longtitude coordinates of each neighborhoods
* Name: Hang Nguyen

In [1]:
# import needed packages
import pandas as pd 
import requests
from bs4 import BeautifulSoup

### 1. Scraping data from neighborhoods in Toronto
That is done in Part 1 of this assignment

In [2]:
# link to scrape information/data from neighborhoods in Toronto 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url).text
soup = BeautifulSoup(data,'html.parser')

# initial list for table content
table_contents=[]

table = soup.find('table')

#extract data from table to list (table_contents)
for row in table.findAll('td'):
    # dictionary called cell having 3 keys PostalCode, Borough and Neighborhood
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        #As postal code contains upto 3 characters extract that using tablerow.p.text
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#Create a dataframe with list
df=pd.DataFrame(table_contents)

# clean the data
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A', 'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',        'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',            'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### 2. Get the latitude and the longitude coordinates of each neighborhood
I will use the csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [3]:
#I download the Geospatial_Coordinates.csv file to my local computer and call it from that
import sys
filepath = sys.path[0] + '/Geospatial_Coordinates.csv' 
df_postalcode = pd.read_csv(filepath)
df_postalcode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
df_postalcode.shape

(103, 3)

Now I have 2 dataframes:
* df: contains data of neighborhoods in Toronto
* df_postalcode: contains postal codes, latitude and Longitude  of each neighborhoods

I will merge 2 dataframes based on column: postal code. In order to merge 2 dataframes based on a column, that column in both dataframes must have same name and same datatypes

In [5]:
# change name of column Postal Code in df_postalcode, so I have same name of Postal Code column in both dataframes
df_postalcode = df_postalcode.rename(columns={'Postal Code':'PostalCode'})
df_postalcode.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
# check datatype of PostalCode in both dataframe
df_postalcode.dtypes

PostalCode     object
Latitude      float64
Longitude     float64
dtype: object

In [7]:
df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [8]:
# merge 2 dataframes based on Postal Code
df_toronto = pd.merge(df,df_postalcode, on = 'PostalCode')

In [9]:
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [10]:
# number of rows in the final dataframe df_toronto
df_toronto.shape

(103, 5)